# Get Raw Images

This notebook will demonstrate how to search for a specific image in order to get its `sequence_id`.

Using a `sequence_id` we will create an `ObservationInfo` object for discovering information about the observation before showing how to download the image metadata and FITS files.

In [1]:
import pandas as pd
from astropy.coordinates import SkyCoord

from panoptes.data.search import search_observations
from panoptes.data.observations import ObservationInfo

## Search for observations

Using a sky coordinate, search for all matching observations with a certain radius.

In [2]:
search_coords = SkyCoord.from_name('M42')
search_coords

<SkyCoord (ICRS): (ra, dec) in deg
    (83.82208, -5.39111)>

In [3]:
search_results = search_observations(
    coords=search_coords,
    radius=10,
    min_num_images=5,
    unit_id='PAN019'
)

print(f'Found {len(search_results)} observations')

Found 19 observations


In [4]:
# Show a random example (run this cell again to see a different entry)
search_results.sample(1).T

,835
camera_id,c623e9
camera_lens_serial_number,RL1552832
camera_serial_number,273074013304
coordinates_mount_dec,-5.39111
coordinates_mount_ra,83.82208
exptime,89.0
field_name,M42
num_images,64.0
sequence_id,PAN019_c623e9_20220117T082323
software_version,POCSv0.8.0.post0.dev7+g8c57c125.dirty


In [5]:
# Show the first and last five entries.
search_results

,camera_id,camera_lens_serial_number,camera_serial_number,coordinates_mount_dec,coordinates_mount_ra,exptime,field_name,num_images,sequence_id,software_version,time,total_exptime,unit_id
861,42433a,RL1548256,273074013260,-5.39111,83.82208,90.000000,M42,65.0,PAN019_42433a_20220114T063812,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-14 06:38:12+00:00,5850.0,PAN019
860,c623e9,RL1552832,273074013304,-5.39111,83.82208,89.000000,M42,64.0,PAN019_c623e9_20220114T063812,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-14 06:38:12+00:00,5696.0,PAN019
858,42433a,RL1548256,273074013260,-5.39111,83.82208,90.000000,M42,90.0,PAN019_42433a_20220114T085722,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-14 08:57:22+00:00,8100.0,PAN019
857,42433a,RL1548256,273074013260,-5.39111,83.82208,89.877193,M42,57.0,PAN019_42433a_20220115T050238,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 05:02:38+00:00,5123.0,PAN019
856,c623e9,RL1552832,273074013304,-5.39111,83.82208,89.000000,M42,56.0,PAN019_c623e9_20220115T050238,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 05:02:38+00:00,4984.0,PAN019
853,c623e9,RL1552832,273074013304,-5.39111,83.82208,89.000000,M42,33.0,PAN019_c623e9_20220115T070915,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 07:09:15+00:00,2937.0,PAN019
852,42433a,RL1548256,273074013260,-5.39111,83.82208,90.000000,M42,33.0,PAN019_42433a_20220115T070915,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 07:09:15+00:00,2970.0,PAN019
851,c623e9,RL1552832,273074013304,-5.39111,83.82208,89.019048,M42,105.0,PAN019_c623e9_20220115T082108,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 08:21:08+00:00,9347.0,PAN019
850,42433a,RL1548256,273074013260,-5.39111,83.82208,90.009615,M42,104.0,PAN019_42433a_20220115T082108,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 08:21:08+00:00,9361.0,PAN019
849,c623e9,RL1552832,273074013304,-5.39111,83.82208,89.000000,M42,65.0,PAN019_c623e9_20220116T045736,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-16 04:57:36+00:00,5785.0,PAN019


## Select observation

A `sequence_id` is used to explore observations. Here we take the most recent observation to get the `sequence_id`.

In [6]:
# We choose an entry that only has a few images while running on binder and get the most recent.
record = search_results.query('num_images < 15').iloc[-1]
record.T

camera_id                                                   c623e9
camera_lens_serial_number                                RL1552832
camera_serial_number                                  273074013304
coordinates_mount_dec                                     -5.39111
coordinates_mount_ra                                      83.82208
exptime                                                       89.0
field_name                                                     M42
num_images                                                    12.0
sequence_id                          PAN019_c623e9_20220117T104417
software_version             POCSv0.8.0.post0.dev7+g8c57c125.dirty
time                                     2022-01-17 10:44:17+00:00
total_exptime                                               1068.0
unit_id                                                     PAN019
Name: 830, dtype: object

## Get Observation info

We can get more information about the observation by creating an `ObservationInfo` object. You can create it with just a `sequence_id` (e.g. `ObservationInfo(sequence_id=record.sequence_id)) but here we pass all the meta data.

In [7]:
obs_info = ObservationInfo(meta=record)

# Show the observation object.
obs_info

Found 12 images in observation


Obs: seq_id=PAN019_c623e9_20220117T104417 num_images=12

In [8]:
obs_info.meta

camera_id                                                   c623e9
camera_lens_serial_number                                RL1552832
camera_serial_number                                  273074013304
coordinates_mount_dec                                     -5.39111
coordinates_mount_ra                                      83.82208
exptime                                                       89.0
field_name                                                     M42
num_images                                                    12.0
sequence_id                          PAN019_c623e9_20220117T104417
software_version             POCSv0.8.0.post0.dev7+g8c57c125.dirty
time                                     2022-01-17 10:44:17+00:00
total_exptime                                               1068.0
unit_id                                                     PAN019
Name: 830, dtype: object

### Show example of the image metadata and how to save to file

The metadata for each image is stored in the `obs_info.image_metadata` dataframe. 

We view a random example and then write the metadata to a file that can be downloaded in the file browser on the left.

In [9]:
# Randomly sample one image from the dataframe then transpose it so it's easier to read.
obs_info.image_metadata.sample(1).T

time,2022-01-17 10:53:36+00:00
received_time,2022-01-17 10:59:06.320000+00:00
uid,PAN019_c623e9_20220117T104417_20220117T105336
status,MATCHED
file_creation_date,2022-01-17 10:53:36+00:00
coordinates_mount_ha,2.722777
coordinates_dec,NaN
coordinates_ra,NaN
coordinates_airmass,1.467624
environment_moonsep,43.605207
environment_moonfrac,0.996112


In [10]:
obs_info.image_metadata.to_csv(f'{obs_info.sequence_id}-image-metadata.csv')

### Download files locally

We can use the built-in `download_images` method of our `obs_info` object to download the files locally.

By default this will create an output directory that is the same as the `sequence_id` but this can be changed. You can also pass a limited list of files for download. 

Run `obs_info.download_images?` for more information.

In [11]:
image_files = obs_info.download_images()

  0%|          | 0/12 [00:00<?, ?it/s]

### Write image list to a CSV file, use `wget` to download

Or, if you prefer you can just get a list of files saved as csv and work on them in a command line terminal.

This first creates a DataFrame then immediately writes that to a CSV file. There are many ways to accomplish this but this is an easy way.

We then view the last few lines of the file using the command line tool `tail` called in a cell with the `!` operator.

We then use the same `!` operator to call `wget` in the shell and download a list of files. Note that you can just copy the CSV file to your computer or server and use the `wget -i <filename.txt>` command to download all the files. **Note that this can be a lot of data!**

In [12]:
# Create a dataframe and then write it to a CSV file.
pd.DataFrame(obs_info.get_image_list()).to_csv(f'{obs_info.sequence_id}-raw.txt', index=False, header=False)

In [13]:
# View the last entries in the file.
!tail {obs_info.sequence_id}-raw.txt

https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T104928.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T105131.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T105336.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T105543.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T105747.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T105958.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T110207.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T110412.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/c623e9/20220117T104417/20220117T110622.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN0

#### Download the data

In [14]:
### Uncomment to run ###
# !wget -i {obs_info.sequence_id}-raw.txt
